In [ ]:
import subprocess
import time

stockfish_path = r"C:\Users\artur\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

engine = subprocess.Popen(
    stockfish_path,
    universal_newlines=True,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    bufsize=1
)

def send_command(cmd, wait=0.1):
    engine.stdin.write(cmd + '\n')
    engine.stdin.flush()
    time.sleep(wait)
    output = []
    while True:
        line = engine.stdout.readline().strip()
        if line == '' or line == 'uciok' or line.startswith('bestmove'):
            break
        output.append(line)
    return output

send_command('uci')

# Optional: set MultiPV to see top N moves at once
send_command('setoption name MultiPV value 20')

# 4. Set your FEN
fen = input("Insert your FEN here: ")
send_command(f'position fen {fen}')

output = send_command('go depth 5', wait=1)

moves = []
for line in output:
    if line.startswith('info') and 'score cp' in line:
        parts = line.split()
        try:
            move_index = parts.index('pv') + 1
            move = parts[move_index]
            score_index = parts.index('score') + 2  # cp value
            score = int(parts[score_index])
            moves.append((move, score))
        except:
            continue

# 7. Find the worst move
if moves:
    worst_move = min(moves, key=lambda x: x[1])  # lowest evaluation for White
    print("Worst move:", worst_move[0], "Score:", worst_move[1])
else:
    print("No moves found")

# 8. Stop the engine when done
engine.stdin.write('quit\n')
engine.stdin.flush()